In [1]:
from ALLCools.mcds import RegionDS
from ALLCools.dmr import call_dms, call_dmr
import glob as glob
import pandas as pd
import numpy as np
import os

In [2]:
brain_regions = ['PFC','AMY','HIP']

In [3]:
cell_types = ['Neuron','Astro-Epen','Vascular','Immune','OPC-Oligo']

In [4]:
celltype = cell_types[0]
brain_region = brain_regions[0]

In [5]:
mc_bulk_dir = f'/data2st1/junyi/merged_allcs/{brain_region}/'
allc_paths = glob.glob(f'{mc_bulk_dir}/*.tsv.gz')

In [6]:
# 3rd column is the orginal id in the wilcox table
df_dar = pd.read_csv(f'/data2st1/junyi/output/motif/ALL_{celltype}_wilcoxon_DAR_lifted_sorted_gene.bed',sep='\t',header=None,index_col=3)
df_dar.columns = ['chr','start','end','chrmm10','startmm10','endmm10','score','strand','gene_name','gene_id','region','distance']
df_pval = pd.read_csv(f'/data2st1/junyi/output/motif/ALL_{celltype}_wilcoxon.csv',index_col=0)
df_concatted= pd.concat([df_pval.loc[df_dar.index],df_dar],axis=1)
df_dar_selected = df_concatted[(df_concatted.pvals<0.05) & (df_concatted.logfoldchanges>0.1)& (df_concatted.pct_nz_group>0.1)]
df_dar_selected['regionmm10'] = df_dar_selected['chr'] + ':' + df_dar_selected['start'].astype(str) + '-' + df_dar_selected['end'].astype(str)
df_dar_selected['region_extended'] = df_dar_selected['chr'] + ':' + (df_dar_selected['start']-5000).astype(str) + '-' + (df_dar_selected['end']+5000).astype(str)

/tmp/ipykernel_2508315/679740978.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dar_selected['regionmm10'] = df_dar_selected['chr'] + ':' + df_dar_selected['start'].astype(str) + '-' + df_dar_selected['end'].astype(str)
/tmp/ipykernel_2508315/679740978.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dar_selected['region_extended'] = df_dar_selected['chr'] + ':' + (df_dar_selected['start']-5000).astype(str) + '-' + (df_dar_selected['end']+5000).astype(str)


In [7]:
allc_table= {}

for allc_path in cell_types:
    #allc_table["/data2st1/junyi/merged_allcs/PFC/"+allc_path] = mc_bulk_dir+allc_path+'.tsv.gz'
    allc_table[allc_path] = mc_bulk_dir+allc_path+'.tsv.gz'


In [8]:
allc_table

{'Neuron': '/data2st1/junyi/merged_allcs/PFC/Neuron.tsv.gz',
 'Astro-Epen': '/data2st1/junyi/merged_allcs/PFC/Astro-Epen.tsv.gz',
 'Vascular': '/data2st1/junyi/merged_allcs/PFC/Vascular.tsv.gz',
 'Immune': '/data2st1/junyi/merged_allcs/PFC/Immune.tsv.gz',
 'OPC-Oligo': '/data2st1/junyi/merged_allcs/PFC/OPC-Oligo.tsv.gz'}

In [9]:
# make a dict, key is sample name, value is allc path
# import pathlib
# allc_table = {
#     allc_path.name.split('.')[0]: str(allc_path)
#     for allc_path in pathlib.Path(mc_bulk_dir).glob(
#         f'{mc_bulk_dir}/*.tsv.gz')
# }
samples = list(allc_table.keys())
allc_paths = list(allc_table.values())
chrom_size_path = '/home/junyichen/code/whole_mouse_brain/wmb/files/mm10.main.chrom.sizes'
output_dir = f'/data2st1/junyi/output/dmr/{brain_region}/{celltype}/'


In [10]:
if os.path.exists(output_dir) == False:
    os.makedirs(output_dir)

In [11]:
input_region = df_dar_selected.sort_values(by='scores',ascending=False).head(100)
input_region.sort_values(['chr','start','end'],inplace=True)
input_region.drop_duplicates(subset=['chr','start','end'],keep='first',inplace=True)
input_region = input_region.regionmm10.values.tolist()  

In [12]:
input_region

['chr1:165075570-165076071',
 'chr1:173390024-173390525',
 'chr10:62101453-62101954',
 'chr10:78001697-78002198',
 'chr10:79780052-79780553',
 'chr10:81472818-81473319',
 'chr10:93275198-93275699',
 'chr10:109010764-109011265',
 'chr11:4946974-4947475',
 'chr11:6065488-6065989',
 'chr11:19835219-19835720',
 'chr11:46235225-46235726',
 'chr11:54833793-54834294',
 'chr11:80504011-80504512',
 'chr11:103666177-103666678',
 'chr12:29872362-29872863',
 'chr12:85436006-85436507',
 'chr12:100265850-100266351',
 'chr12:104415539-104416040',
 'chr13:24576626-24577127',
 'chr13:43230178-43230679',
 'chr13:55483567-55484068',
 'chr13:59073747-59074248',
 'chr14:8208863-8209364',
 'chr14:31268412-31268913',
 'chr14:55108488-55108989',
 'chr14:65266337-65266838',
 'chr14:66319547-66320048',
 'chr14:69994777-69995278',
 'chr14:70628774-70629275',
 'chr14:76760656-76761157',
 'chr15:77245336-77245837',
 'chr15:100923376-100923877',
 'chr16:13197259-13197760',
 'chr16:16930866-16931367',
 'chr17:732718

In [13]:
call_dms(
    output_dir=output_dir,
    allc_paths=allc_paths,
    samples=samples,
    chrom_size_path=chrom_size_path,
    cpu=16,
    max_row_count=50,
    n_permute=3000,
    min_pvalue=0.01,
    # here we just calculate some small regions for demo
    # do not provide region parameter if you want to run DMR calling for the whole genome
    # This parameter can also be used for call DMR/DMS in specific region of interest
    region=input_region)


RMS tests for 286 sites.RMS tests for 274 sites.RMS tests for 280 sites.
RMS tests for 393 sites.


RMS tests for 259 sites.RMS tests for 262 sites.

RMS tests for 283 sites.
RMS tests for 280 sites.RMS tests for 372 sites.

RMS tests for 238 sites.RMS tests for 310 sites.
RMS tests for 252 sites.

RMS tests for 332 sites.
RMS tests for 255 sites.
RMS tests for 300 sites.RMS tests for 259 sites.

RMS tests for 261 sites.
RMS tests for 272 sites.
RMS tests for 250 sites.
RMS tests for 247 sites.
RMS tests for 264 sites.
RMS tests for 290 sites.
RMS tests for 283 sites.
RMS tests for 283 sites.
RMS tests for 255 sites.
RMS tests for 266 sites.
RMS tests for 265 sites.
RMS tests for 288 sites.
RMS tests for 297 sites.
RMS tests for 302 sites.
RMS tests for 256 sites.
RMS tests for 250 sites.
RMS tests for 272 sites.RMS tests for 280 sites.

RMS tests for 252 sites.
RMS tests for 242 sites.
RMS tests for 304 sites.
RMS tests for 296 sites.
RMS tests for 281 sites.
RMS tests for 302 sites.


In [14]:
call_dmr(output_dir=output_dir,
         p_value_cutoff=0.001,
         frac_delta_cutoff=0.3,
         max_dist=250,
         residual_quantile=0.7,
         corr_cutoff=0.3,
         cpu=16)

ContainsGroupError: path "path '' contains a group" contains a group

In [ ]:
dmr_ds = RegionDS.open(output_dir)
dmr_ds


In [ ]:
df_dmr = pd.DataFrame(
    {
        'chr': dmr_ds['dmr_chrom'].to_numpy(),
        'start': dmr_ds['dmr_start'].to_numpy(),
        'end': dmr_ds['dmr_end'].to_numpy(),
    }
)

In [ ]:
sample_mapping = {}
for i in range(len(dmr_ds['sample'].to_numpy())):
    sample_mapping[i] = dmr_ds['sample'].to_numpy()[i]
sample_mapping

In [ ]:
drm_state = dmr_ds['dmr_state'].to_numpy()
for i in range(len(drm_state)):
    df_dmr[sample_mapping[i]] = drm_state[i]

In [ ]:
df_dmr.to_csv(f'{output_dir}/dmr_stat.csv',index=False)

In [ ]:
np.save(f'{output_dir}/dmr_frac.npy', dmr_ds['dmr_da_frac'].to_numpy())
